In [1]:
import pandas as pd

In [2]:
import sys
sys.path.insert(0, "../")

In [3]:
df = pd.read_csv('../three_grams.csv')
df.head(10)

,token_1,ws,token_2,count
0,Comment,1_NL,Comment,3282
1,Comment,1_NL,package,492
2,package,1_SP,Identifier,2083
3,Identifier,0_SP,.,135330
4,.,0_SP,Identifier,146617
5,Identifier,0_SP,;,28148
6,;,1_NL,EOF,4
7,;,2_NL,import,2961
8,import,1_SP,Identifier,15630
9,;,1_NL,import,13795


In [4]:
count_df = df.loc[:,['token_1', 'token_2', 'count']].groupby(['token_1', 'token_2']).agg(['count', 'sum'])

In [5]:
count = count_df['count']['count']
good_for_insertion = list(count[count>=2].index)

In [6]:
good_for_insertion_index = []
for index, (token_1, token_2) in df.loc[:,['token_1', 'token_2']].iterrows():
    if (token_1, token_2) in good_for_insertion:
        good_for_insertion_index.append(index)

In [36]:
good_for_insertion_df = df.iloc[good_for_insertion_index ,:]

In [37]:
good_for_insertion_set = set(
    map(
        lambda line: (line[1][0], line[1][1]),
        df.loc[:,['token_1', 'token_2']].iterrows()
    )
)
def is_good_for_insertion(token_a, token_b):
    return (token_a, token_b) in good_for_insertion_set

In [56]:
def list_alternatives_with_probability(token_a, ws, token_b):
    df_token_1 = good_for_insertion_df[good_for_insertion_df.token_1 == token_a]
    df_token_1_and_2 = df_token_1[df_token_1.token_2 == token_b]
    final = df_token_1_and_2[df_token_1_and_2.ws != ws]
    total = final['count'].sum()
    return {
        ws:count/total
        for _, (_, ws, _, count) in final.iterrows()
    }

In [57]:
list_alternatives_with_probability('Comment', '1_NL', 'Comment')

{'2_NL': 0.6451612903225806,
 '2_NL_4_ID': 0.13978494623655913,
 '1_NL_8_ID': 0.010752688172043012,
 '3_NL': 0.010752688172043012,
 '1_NL_16_DD': 0.010752688172043012,
 '1_NL_4_DD': 0.053763440860215055,
 '2_NL_4_DD': 0.03225806451612903,
 '1_NL_8_DD': 0.03225806451612903,
 '2_NL_8_ID': 0.010752688172043012,
 '2_NL_8_DD': 0.010752688172043012,
 '1_NL_12_ID': 0.010752688172043012,
 '1_NL_2_ID': 0.010752688172043012,
 '1_NL_50_ID': 0.021505376344086023}

In [9]:
import java_lang_utils as jlu
from core import *

In [10]:
from ml import get_token_value, get_space_value

In [11]:
from Corpus import Corpus

In [12]:
corpus = Corpus('../corpora/repairnator-corpus/', 'repairnator')

In [13]:
tokenizer = jlu.Tokenizer()

In [14]:
file_dir = corpus.files[10][2]
file_dir

'../corpora/repairnator-corpus/data/51/MavenHelper.java'

In [15]:
original_source = open_file(file_dir)

In [21]:
def modify_source(source):
    tokenized_source = tokenizer.tokenize(source)
    for index, (token_a, ws, token_b) in tokenized_source.enumerate_3_grams():
        if is_good_for_insertion(get_token_value(token_a), get_token_value(token_b)):
            print(index)

In [22]:
modify_source(original_source)